In [51]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024')
from llm import LM
import importlib
importlib.reload(sys.modules['llm'])
from llm import LM
import gc
import torch
from accelerate.utils import release_memory

def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

In [52]:

# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"

# empty the memory
flush()
# or
# release_memory()

# llm = LM(
#     model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     tokenizer_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     padding_side="left",
#     dtype="bf16",
#     device_map= "auto",
#     use_flash_attention_2=False,
#     access_token=None,
#     cache_dir=cache_dir,
#     accelerator = None
# )
llm = LM(
    model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    padding_side="left",
    dtype="bf16",
    device_map= "auto",
    attn_implementation="flash_attention_2",
    #use_flash_attention_2=False, (deprecated)
    access_token=None,
    cache_dir=cache_dir,
    accelerator = None
)



/data/rech/huiyuche/envs/trec_ikat/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


In [ ]:
def bytes_to_giga_bytes(bytes):
    return bytes / 1024 / 1024 / 1024

bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

In [63]:
print(type(llm.model))

<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


In [57]:
messages = [
    {"role": "user", "content": "what is the capital of France?"},
]
response = llm.hf_llm_generate(
    messages,
    temperature = 1,
    top_p = 0.9,
    max_new_tokens = 256,
    do_sample = True,
    num_beams = 1,
    num_return_sequences = 2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [58]:
print(response)

['The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is also home to numerous cultural institutions, beautiful parks, and excellent cuisine. Paris has a rich history and is a popular tourist destination for people all over the world.', 'The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its art, culture, and history. Some of its most famous landmarks include the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. Paris is also home to many renowned universities and research institutions.']


In [61]:
messages = [
    [{"role": "user", "content": "Who are you?"}],
    [{"role": "user", "content": "what is the capital of Germany?"}],
]
outputs = llm.hf_llm_generate_via_pipline(
    messages,
    temperature = 0.1,
    top_p = 1,
    max_new_tokens = 256,
    do_sample = True,
    num_beams = 1,
    num_return_sequences = 2
)

In [62]:
print(outputs)

[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative text, and assist with various tasks."}]}, {'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."}]}], [{'generated_text': [{'role': 'user', 'content': 'what is the capital of Germany?'}, {'role': 'assistant', 'content': ' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the

In [65]:
[[alternative["generated_text"][-1]["content"] for alternative in output] for output in outputs ]

[[" I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative text, and assist with various tasks.",
  " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."],
 [' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin is home to many famous landmarks, including the Brandenburg Gate, the Reichstag Building, and the Berlin Wall Memorial.',
  " The capital city of Germany is B

In [11]:
from accelerate import Accelerator
from accelerate import init_empty_weights
from accelerate import load_checkpoint_and_dispatch

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    logging
)


# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct", cache_dir=cache_dir
)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", cache_dir = cache_dir, device_map="auto")

# with init_empty_weights():
#     model = AutoModelForCausalLM()

# model = load_checkpoint_and_dispatch(
#     model,
#     checkpoint=cache_dir + "/models--meta-llama--Meta-Llama-3-8B-Instruct",
#     device_map="auto",
# )
# print(model)
print(model)

accelerator = Accelerator()
print(accelerator.device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.71s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [ ]:
del model
del pipe

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': "I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my knowledge based on my training data.\n\nI'm a large language model, which means I've been trained on a massive dataset of text from various sources, including books, articles, and websites. This training enables me to understand and generate human-like language, allowing me to engage in conversations, answer questions, and even create text based on a given prompt.\n\nI'm constantly learning and improving my responses based on the interactions I have with users like you. So, feel free to ask me anything, and I'll do my best to provide a helpful and accurate response!"}]}]

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a physical presence. I exist solely to process and generate text."}]}, {'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."}]}]]

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative text, and assist with various tasks."}]}, {'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."}]}], [{'generated_text': [{'role': 'user', 'content': 'what is the capital of Germany?'}, {'role': 'assistant', 'content': ' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin is home to many famous landmarks, including the Brandenburg Gate, the Reichstag Building, and the Berlin Wall Memorial.'}]}, {'generated_text': [{'role': 'user', 'content': 'what is the capital of Germany?'}, {'role': 'assistant', 'content': " The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin has been the capital of Germany since the country's reunification in 1990."}]}]]